In [1]:
# Imports
# Basics
import pandas as pd 
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline

# gensim
import gensim

# keras
np.random.seed(13)
from keras.models import Sequential
from keras.layers import (Dense, Embedding, Reshape, Activation, 
                          SimpleRNN, LSTM, Convolution1D, 
                          MaxPooling1D, Dropout, Bidirectional)
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop


# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import keras
from keras.datasets import reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)
word_index = reuters.get_word_index(path="reuters_word_index.json")

print('# of Training Samples: {}'.format(len(x_train)))
print('# of Test Samples: {}'.format(len(x_test)))

num_classes = max(y_train) + 1
print('# of Classes: {}'.format(num_classes))
# of Training Samples: 8982
# of Test Samples: 2246
# of Classes: 46
index_to_word = {}
for key, value in word_index.items():
    index_to_word[value] = key
print(' '.join([index_to_word[x] for x in x_train[0]]))
print(y_train[0])

# of Training Samples: 8982
# of Test Samples: 2246
# of Classes: 46
the wattie nondiscriminatory mln loss for plc said at only ended said commonwealth could 1 traders now april 0 a after said from 1985 and from foreign 000 april 0 prices its account year a but in this mln home an states earlier and rise and revs vs 000 its 16 vs 000 a but 3 psbr oils several and shareholders and dividend vs 000 its all 4 vs 000 1 mln agreed largely april 0 are 2 states will billion total and against 000 pct dlrs
3


C:\Users\DELL\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\DELL\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [3]:
print(' '.join([index_to_word[x] for x in x_train[2]]))

the lt dlrs demand 000 reuter dividend year lt plus billion 04 000 reuter dividend year an worth new vs reuter dlrs warburg on shrs earnings countries new vs reuter 1985 billion vs 2 lt 4 division 000 reuter from go 000 lt plus which mid 000 reuter from total 000 an 71 billion vs reuter dlr also vs shrs earnings countries 4 vs reuter 1985 from vs some now april 0 related in corp it inc strong cents dollar were after april 0 crisis or ontario more index 10 electric company taking report it in estimated but trading texas said united said came a advising up said countries vs 000 3 delayed central said which objections on future 617 said came a includes refile profit said meeting trade vs 3 supplie up said 1985 were vs pct dlrs


# RNN: Text Classification


In [14]:
max_features = 2000
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=max_features)

In [15]:
# Same data loading as before
max_features = 2000
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=max_features)
maxlen = 10
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
nb_epoch = 20

In [18]:
X_train.shape

(8982, 10)

In [23]:
# Create the model
rnn_model = Sequential()
# The Embedding layer allows us to map words into dense vectors as inputs, common first layer
rnn_model.add(Embedding(input_dim=max_features, output_dim=100, embeddings_initializer='glorot_uniform', input_length=maxlen))
# This is the most basic kind of RNN!  We're using 20 units, 
#which somewhat reflects our "memory" of past events in a sequence
# For the purposes of keras, it's just another type of "unit" you can try!
rnn_model.add(SimpleRNN(50, return_sequences=False))
rnn_model.add(Dense(100))
rnn_model.add(Activation('relu'))
rnn_model.add(Dense(46))
rnn_model.add(Activation('softmax'))

rnn_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 100)           200000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 50)                7550      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               5100      
_________________________________________________________________
activation_4 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 46)                4646      
_________________________________________________________________
activation_5 (Activation)    (None, 46)                0         
Total params: 217,296
Trainable params: 217,296
Non-trainable params: 0
________________________________________________

In [25]:
(100 input vector + 20 units) * 20 units + 20 biases



7550

In [24]:
# Train the model
rnn_model.fit(X_train, y_train, batch_size=256, epochs=500, 
              validation_data=(X_test, y_test))

Epoch 1/500
36/36 [==============================] - 2s 16ms/step - loss: 3.8097 - accuracy: 0.0243 - val_loss: 3.8082 - val_accuracy: 0.0289
Epoch 2/500
36/36 [==============================] - 0s 8ms/step - loss: 3.8068 - accuracy: 0.0288 - val_loss: 3.8068 - val_accuracy: 0.0316
Epoch 3/500
36/36 [==============================] - 0s 9ms/step - loss: 3.8064 - accuracy: 0.0273 - val_loss: 3.8054 - val_accuracy: 0.0334
Epoch 4/500
36/36 [==============================] - 0s 8ms/step - loss: 3.8047 - accuracy: 0.0292 - val_loss: 3.8039 - val_accuracy: 0.0374
Epoch 5/500
36/36 [==============================] - 0s 9ms/step - loss: 3.8034 - accuracy: 0.0293 - val_loss: 3.8024 - val_accuracy: 0.0401
Epoch 6/500
36/36 [==============================] - 0s 9ms/step - loss: 3.8016 - accuracy: 0.0339 - val_loss: 3.8009 - val_accuracy: 0.0441
Epoch 7/500
36/36 [==============================] - 0s 10ms/step - loss: 3.8007 - accuracy: 0.0320 - val_loss: 3.7993 - val_accuracy: 0.0467
Epoch 8/500

Epoch 59/500
36/36 [==============================] - 0s 9ms/step - loss: 3.5783 - accuracy: 0.3565 - val_loss: 3.5694 - val_accuracy: 0.3602
Epoch 60/500
36/36 [==============================] - 0s 9ms/step - loss: 3.5692 - accuracy: 0.3469 - val_loss: 3.5543 - val_accuracy: 0.3598
Epoch 61/500
36/36 [==============================] - 0s 9ms/step - loss: 3.5537 - accuracy: 0.3442 - val_loss: 3.5378 - val_accuracy: 0.3624
Epoch 62/500
36/36 [==============================] - 0s 10ms/step - loss: 3.5336 - accuracy: 0.3519 - val_loss: 3.5198 - val_accuracy: 0.3638
Epoch 63/500
36/36 [==============================] - 0s 9ms/step - loss: 3.5150 - accuracy: 0.3477 - val_loss: 3.5002 - val_accuracy: 0.3629
Epoch 64/500
36/36 [==============================] - 0s 10ms/step - loss: 3.4977 - accuracy: 0.3463 - val_loss: 3.4788 - val_accuracy: 0.3620
Epoch 65/500
36/36 [==============================] - 0s 9ms/step - loss: 3.4766 - accuracy: 0.3552 - val_loss: 3.4551 - val_accuracy: 0.3615
Epoc

36/36 [==============================] - 0s 8ms/step - loss: 2.4953 - accuracy: 0.3518 - val_loss: 2.4943 - val_accuracy: 0.3620
Epoch 174/500
36/36 [==============================] - 0s 8ms/step - loss: 2.4890 - accuracy: 0.3511 - val_loss: 2.4937 - val_accuracy: 0.3620
Epoch 175/500
36/36 [==============================] - 0s 8ms/step - loss: 2.4835 - accuracy: 0.3547 - val_loss: 2.4930 - val_accuracy: 0.3620
Epoch 176/500
36/36 [==============================] - 0s 8ms/step - loss: 2.5281 - accuracy: 0.3460 - val_loss: 2.4923 - val_accuracy: 0.3620
Epoch 177/500
36/36 [==============================] - 0s 8ms/step - loss: 2.4840 - accuracy: 0.3506 - val_loss: 2.4917 - val_accuracy: 0.3620
Epoch 178/500
36/36 [==============================] - 0s 8ms/step - loss: 2.4784 - accuracy: 0.3595 - val_loss: 2.4910 - val_accuracy: 0.3620
Epoch 179/500
36/36 [==============================] - 0s 8ms/step - loss: 2.4952 - accuracy: 0.3475 - val_loss: 2.4904 - val_accuracy: 0.3620
Epoch 180/500

36/36 [==============================] - 0s 10ms/step - loss: 2.4243 - accuracy: 0.3512 - val_loss: 2.4309 - val_accuracy: 0.3620
Epoch 288/500
36/36 [==============================] - 0s 9ms/step - loss: 2.4231 - accuracy: 0.3478 - val_loss: 2.4304 - val_accuracy: 0.3620
Epoch 289/500
36/36 [==============================] - 0s 9ms/step - loss: 2.4022 - accuracy: 0.3540 - val_loss: 2.4299 - val_accuracy: 0.3620
Epoch 290/500
36/36 [==============================] - 0s 9ms/step - loss: 2.3988 - accuracy: 0.3596 - val_loss: 2.4293 - val_accuracy: 0.3620
Epoch 291/500
36/36 [==============================] - 0s 9ms/step - loss: 2.4173 - accuracy: 0.3512 - val_loss: 2.4288 - val_accuracy: 0.3620
Epoch 292/500
36/36 [==============================] - 0s 8ms/step - loss: 2.4217 - accuracy: 0.3515 - val_loss: 2.4282 - val_accuracy: 0.3620
Epoch 293/500
36/36 [==============================] - 0s 9ms/step - loss: 2.4125 - accuracy: 0.3565 - val_loss: 2.4276 - val_accuracy: 0.3620
Epoch 294/50

36/36 [==============================] - 0s 10ms/step - loss: 2.3619 - accuracy: 0.3519 - val_loss: 2.3582 - val_accuracy: 0.3629
Epoch 402/500
36/36 [==============================] - 0s 10ms/step - loss: 2.3297 - accuracy: 0.3596 - val_loss: 2.3575 - val_accuracy: 0.3624
Epoch 403/500
36/36 [==============================] - 0s 9ms/step - loss: 2.3234 - accuracy: 0.3584 - val_loss: 2.3568 - val_accuracy: 0.3624
Epoch 404/500
36/36 [==============================] - 0s 9ms/step - loss: 2.3395 - accuracy: 0.3532 - val_loss: 2.3561 - val_accuracy: 0.3633
Epoch 405/500
36/36 [==============================] - 0s 9ms/step - loss: 2.3772 - accuracy: 0.3439 - val_loss: 2.3554 - val_accuracy: 0.3638
Epoch 406/500
36/36 [==============================] - 0s 9ms/step - loss: 2.3900 - accuracy: 0.3396 - val_loss: 2.3547 - val_accuracy: 0.3638
Epoch 407/500
36/36 [==============================] - 0s 9ms/step - loss: 2.3358 - accuracy: 0.3619 - val_loss: 2.3540 - val_accuracy: 0.3638
Epoch 408/5

# LSTM Example: Sentiment Analysis

Here is some code to train sentiment analysis on IMDB reviews.


In [5]:
# Set parameters
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

# Load data (Keras utility)
print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

# Pad Short sentences
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

# Build our model!
print('Build model...')
lstm_model = Sequential()
lstm_model.add(Embedding(max_features, 128))
# Here's the LSTM magic!
lstm_model.add(LSTM(128, recurrent_dropout=.2))  # try using a GRU instead, for fun
lstm_model.add(Dense(1))
# Sigmoid for binary classification
lstm_model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
lstm_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
lstm_model.fit(X_train, y_train, batch_size=batch_size, epochs=2,
               validation_data=(X_test, y_test))
score, acc = lstm_model.evaluate(X_test, y_test,
                                 batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)
Build model...
Train...
Epoch 1/2
782/782 [==============================] - 110s 136ms/step - loss: 0.5198 - accuracy: 0.7234 - val_loss: 0.3671 - val_accuracy: 0.8346
Epoch 2/2
782/782 [==============================] - 13s 17ms/step - loss: 0.3758 - accuracy: 0.8299
Test score: 0.37582284212112427
Test accuracy: 0.8299199938774109


# CNN and LSTM: Sentiment Analysis
-  

In [6]:
# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
filter_length = 5
nb_filter = 64
pool_length = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
nb_epoch = 2
'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

# Load data
print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')



# Pad sentences
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 100)
X_test shape: (25000, 100)


In [7]:

from tensorflow.keras.layers import Conv1D 

print('Build model...')

cnn_model = Sequential()
cnn_model.add(Embedding(max_features, embedding_size, input_length=maxlen))
cnn_model.add(Dropout(0.25))

# Convolution!
#Conv1D(filters=100, kernel_size=4

cnn_model.add(Conv1D(filters=nb_filter,
                            kernel_size=filter_length,
                            #border_mode='valid',
                            activation='relu' ))
                            #subsample_length=1
                   
#cnn_model.add(MaxPooling1D(pool_length=pool_length))

# LSTM!
cnn_model.add(LSTM(lstm_output_size))
cnn_model.add(Dense(1))
cnn_model.add(Activation('sigmoid'))

cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
cnn_model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
          validation_data=(X_test, y_test))
score, acc = cnn_model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Epoch 1/2
834/834 [==============================] - 72s 84ms/step - loss: 0.5150 - accuracy: 0.7197 - val_loss: 0.3311 - val_accuracy: 0.8565
Epoch 2/2
834/834 [==============================] - 13s 15ms/step - loss: 0.3722 - accuracy: 0.8488
Test score: 0.3722308576107025
Test accuracy: 0.8487600088119507


# BRNN: Sentiment Analysis
-

In [8]:
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

brnn_model = Sequential()
brnn_model.add(Embedding(max_features, 128, input_length=maxlen))

# Bidirectional LSTM!!!
brnn_model.add(Bidirectional(LSTM(64)))
brnn_model.add(Dropout(0.5))
brnn_model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
brnn_model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
brnn_model.fit(X_train, y_train,
               batch_size=batch_size,
               epochs=4,
               validation_data=[X_test, y_test])

score, acc = brnn_model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...


2022-09-17 02:49:32,270 : WARNING : The `nb_words` argument in `load_data` has been renamed `num_words`.


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 100)
X_test shape: (25000, 100)
Train...
Epoch 1/4
782/782 [==============================] - 70s 81ms/step - loss: 0.5156 - accuracy: 0.7207 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/4
782/782 [==============================] - 65s 83ms/step - loss: 0.2252 - accuracy: 0.9152 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/4
782/782 [==============================] - 62s 79ms/step - loss: 0.1212 - accuracy: 0.9567 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/4
782/782 [==============================] - 18s 22ms/step - loss: 0.6233 - accuracy: 0.8306
Test score: 0.6232659816741943
Test accuracy: 0.8306400179862976


In [9]:
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
dropout (Dropout)            (None, 100, 128)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 64)            41024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 70)                37800     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 71        
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 2,638,895
Trainable params: 2,638,895
Non-trainable params: 0
____________________________________________

In [10]:
brnn_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,658,945
Trainable params: 2,658,945
Non-trainable params: 0
_________________________________________________________________


# Save Trained Models

In [ ]:
import pickle


with open('models/rnn_model.pkl', 'wb') as pkl:
    pickle.dump(rnn_model, pkl)
with open('models/lstm_model.pkl', 'wb') as pkl:
    pickle.dump(lstm_model, pkl)
with open('models/cnn_model.pkl', 'wb') as pkl:
    pickle.dump(cnn_model, pkl)
with open('models/brnn_model.pkl', 'wb') as pkl:
    pickle.dump(brnn_model, pkl)

